## Small Oscillations Double Pendulum

##Preamble

In [1]:
from sympy import symbols, Function, sin, cos, Rational, diff, simplify, zeros, Inverse, expand, Transpose, sqrt, Matrix, N

Symbolic variables

In [2]:
t,l1,l2,m1,m2,g=symbols('t,l1,l2,m1,m2,g',positive=True)

Generalized coordinates

In [3]:
phi1,phi2=Function('phi1')(t),Function('phi2')(t)

Cartesian Coordinates

In [4]:
x1=l1*sin(phi1)
x2=l1*sin(phi1)+l2*sin(phi2)
y1=l1*cos(phi1)
y2=l1*cos(phi1)+l2*cos(phi2)

## Kinetic Energy $T$

In [5]:
T=(Rational(1,2)*m1*((x1.diff(t))**2+(y1.diff(t))**2)+Rational(1,2)*m2*((x2.diff(t))**2+(y2.diff(t))**2)).simplify()
T

l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## Potential Energy $U$ (notice that $y$ increases downwards)

In [6]:
U=(-m1*g*y1-m2*g*y2).simplify()
U

-g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t)))

## Lagrangian $L=T-U$

In [7]:
Lag=(T-U).simplify()
Lag

g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t))) + l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## For concretness, in the following we will focus on a double pendulum with $l_1=2l$, $l_2=l$, $m_1=m$, $m_2=2m$

In [8]:
l,m=symbols('l,m',positive=True)

## Inertia Tensor

In [9]:
dot_phi=(phi1.diff(t),phi2.diff(t))
M0=zeros(len(dot_phi))
for i in range(len(dot_phi)):
  for j in range(len(dot_phi)):
    M0[i,j]=simplify((T.diff(dot_phi[i])).diff(dot_phi[j]).subs([(phi1,0),(phi2,0)]))
M0

Matrix([
[l1**2*(m1 + m2), l1*l2*m2],
[       l1*l2*m2, l2**2*m2]])

In [10]:
M=simplify(M0.subs([(l1,2*l),(l2,l),(m1,m),(m2,2*m)]))
M

Matrix([
[12*l**2*m, 4*l**2*m],
[ 4*l**2*m, 2*l**2*m]])

## Harmonic Tensor

In [11]:
phi=(phi1,phi2)
K0=zeros(len(phi))
for i in range(len(phi)):
  for j in range(len(phi)):
    K0[i,j]=simplify((U.diff(phi[i])).diff(phi[j]).subs([(phi1,0),(phi2,0)]))
K0

Matrix([
[g*l1*(m1 + m2),       0],
[             0, g*l2*m2]])

In [12]:
K=simplify(K0.subs([(l1,2*l),(l2,l),(m1,m),(m2,2*m)]))
K

Matrix([
[6*g*l*m,       0],
[      0, 2*g*l*m]])

Step 1.- Diagonalize the Inertia Tensor

The method **diagonalize** returns the modal matrix $T$ thad diagonalizes the input matrix $M$ rendering a diagonal matrix $M_D$ according to $T^{-1}MT=M_D$

In [13]:
T,MD=M.diagonalize()

Orthonormalization of the eigenvectors in the modal matrix $T$ that produces an orthogonal modal matrix $O_{M}$ such that $O_{M}^T M O_{M}=M_D$

In [14]:
OM=simplify(T*(sqrt((Transpose(T)*T)**(-1))))

Verification of orthonormality and diagonalization

In [15]:
simplify(Transpose(OM)*OM)

Matrix([
[1, 0],
[0, 1]])

In [16]:
simplify(Transpose(OM)*M*OM-MD)

Matrix([
[0, 0],
[0, 0]])

Transformed Harmonic Tensor

In [17]:
KM=simplify(Transpose(OM)*K*OM)
KM

Matrix([
[-10*sqrt(41)*g*l*m/41 + 4*g*l*m,           -8*sqrt(41)*g*l*m/41],
[           -8*sqrt(41)*g*l*m/41, 10*sqrt(41)*g*l*m/41 + 4*g*l*m]])

Step 2.- Diagonalization of the Rescaled Harmonic Tensor

Rescaled Harmonic Tensor

In [18]:
W=simplify(sqrt(MD)**(-1)*KM*sqrt(MD)**(-1))
W

Matrix([
[2*g*(82 - 5*sqrt(41))/(41*l*(7 - sqrt(41))),                        -2*sqrt(82)*g/(41*l)],
[                       -2*sqrt(82)*g/(41*l), 2*g*(5*sqrt(41) + 82)/(41*l*(sqrt(41) + 7))]])

In [19]:
Q,WD=W.diagonalize()

Orthonormalization of the eigenvectors in the modal matrix $Q$ that produces an orthogonal modal matrix $O_{W}$ such that $O_{W}^T W O_{W}=W_D$

In [20]:
OW=simplify(Q*(sqrt((Transpose(Q)*Q)**(-1))))
OW

Matrix([
[(-47*sqrt(2) + sqrt(4674))/(2*sqrt(2337 - 47*sqrt(2337))), (-sqrt(4674) - 47*sqrt(2))/(2*sqrt(47*sqrt(2337) + 2337))],
[                             8/sqrt(2337 - 47*sqrt(2337)),                              8/sqrt(47*sqrt(2337) + 2337)]])

Verification of orthonormality and diagonalization

In [21]:
simplify(Transpose(OW)*OW)

Matrix([
[1, 0],
[0, 1]])

In [22]:
simplify(Transpose(OW)*W*OW-WD)

Matrix([
[0, 0],
[0, 0]])

Squared Eigenfrequencies

In [23]:
WD

Matrix([
[g*(9/4 - sqrt(57)/4)/l,                      0],
[                     0, g*(sqrt(57)/4 + 9/4)/l]])

In [24]:
N(WD)

Matrix([
[0.362541391182313*g/l,                    0],
[                    0, 4.13745860881769*g/l]])

Modal Matrix

In [25]:
S=simplify(OM*sqrt(MD)**(-1)*OW)
S

Matrix([
[(-(5 - sqrt(41))*(-sqrt(4674) + 47*sqrt(2))*sqrt(-141*sqrt(2337) - 893*sqrt(57) + 1083*sqrt(41) + 7011) + 16*(5 + sqrt(41))*sqrt(-1083*sqrt(41) - 141*sqrt(2337) + 893*sqrt(57) + 7011))/(16*l*sqrt(m)*(2337 - 47*sqrt(2337))), (16*(5 + sqrt(41))*sqrt(-1083*sqrt(41) - 893*sqrt(57) + 141*sqrt(2337) + 7011) - (5 - sqrt(41))*(47*sqrt(2) + sqrt(4674))*sqrt(893*sqrt(57) + 141*sqrt(2337) + 1083*sqrt(41) + 7011))/(16*l*sqrt(m)*(47*sqrt(2337) + 2337))],
[                                (16*sqrt(-1083*sqrt(41) - 141*sqrt(2337) + 893*sqrt(57) + 7011) - (-sqrt(4674) + 47*sqrt(2))*sqrt(-141*sqrt(2337) - 893*sqrt(57) + 1083*sqrt(41) + 7011))/(4*l*sqrt(m)*(2337 - 47*sqrt(2337))),                               (-(47*sqrt(2) + sqrt(4674))*sqrt(893*sqrt(57) + 141*sqrt(2337) + 1083*sqrt(41) + 7011) + 16*sqrt(-1083*sqrt(41) - 893*sqrt(57) + 141*sqrt(2337) + 7011))/(4*l*sqrt(m)*(47*sqrt(2337) + 2337))]])

In [26]:
N(S)

Matrix([
[0.205467096352297/(l*m**0.5), 0.455832504673115/(l*m**0.5)],
[0.233710317574675/(l*m**0.5), -1.20223936362904/(l*m**0.5)]])

Normal coordinates

In [27]:
NC=Inverse(S)*Matrix([[phi1], [phi2]])

In [28]:
N(NC)

Matrix([
[  3.40044642652648*l*m**0.5*phi1(t) + 1.28928902055865*l*m**0.5*phi2(t)],
[0.661032601561205*l*m**0.5*phi1(t) - 0.581148708565628*l*m**0.5*phi2(t)]])